**Important note**! Before you turn in this lab notebook, make sure everything runs as expected:

- First, restart the kernel -- in the menubar, select Kernel → Restart.
- Then run all cells -- in the menubar, select Cell → Run All.

Make sure you fill in any place that says YOUR CODE HERE or "YOUR ANSWER HERE."

# Data Wrangling for Machine Learning

In this exercise we will take you through the basics of data cleaning that often is the majority of your work before fitting a training a machine learning model.

Data often has a lot of missing values, incorrect data types, rows that need to be removed etc., and this problemgives you a flavor of what is often required before any sort of descriptive, predictive, or prescriptive analysis.

For this exercise, we will be using a dataset with credit approval scores.

In [1]:
import pandas as pd
import numpy as np
import re

pd.options.mode.chained_assignment = None

In [55]:
credit = pd.read_csv('creditapproval.csv')

Let us take a look at the data.

In [56]:
print('The dataset has {} rows.'.format(len(credit))) 
credit.head()

The dataset has 624 rows.


,Predictor A,Predictor B,Predictor C,Predictor D,Predictor E,Predictor F,Predictor G,Response
0,30.83,G,1.25,1,t,202.0,Mrketing 1,1.0
1,58.67,J,3.04,6,t,43.0,Mkt6,1.0
2,24.50,B,1.50,0,f,280.0,M0,1.0
3,27.83,B,3.75,5,f,100.0,Marketing 5,1.0
4,20.17,B,1.71,0,t,120.0,Spend 0,1.0


The data hence has 7 predictors and 1 response variable.

In machine learning or predictive modelling in general, you use predictors (in this case 7 of them) to predict its corresponding response.

But before we can move onto predictive modelling, we need to clean the data. Data cleaning often is the most important part of machine learning and we exactly going to do that bit.

For instance, let's check for columns having missing values.

In [57]:
credit.isnull().any()

Predictor A     True
Predictor B     True
Predictor C    False
Predictor D    False
Predictor E    False
Predictor F     True
Predictor G    False
Response        True
dtype: bool

So the columns **Predictor A, Predictor B, Predictor F, and Response** all have missing values. You'll treat these cases through the exercises below.

Data without a **Response variable** can neither be used in training the model or in testing it. We would hence like to remove rows that have **both Predictor F** and **Response as NaN values**


**Exercise 0** (1 point): Create a new dataframe named **`creditwithresponse`** that is a copy of **`credit`** but with any rows missing **either** `Predictor F` **or** `Response` removed.

In [58]:
creditwithresponse = credit.copy()
creditwithresponse = creditwithresponse[np.isfinite(creditwithresponse.Response) |
                                        np.isfinite(creditwithresponse['Predictor F'])]


In [59]:
##Test cell: Exercise 0
assert len(creditwithresponse) == 612, "The length of your newly created dataframe does not match the solution"
assert len(creditwithresponse[creditwithresponse['Predictor F'].isnull()]) == 0, "Some NaN values still exist in your new dataframe."
assert len(creditwithresponse[creditwithresponse['Response'].isnull()]) == 0, "Almost there! Though some NaN values still exist in your new dataframe."

print("\n(Passed!)")


(Passed!)


What about the other predictors?

One technique is to replace missing values with sensible substitutes. For instance, we might replace a missing value with the **mean** of the remaining values in the case of a numerical variable, or the **mode** in the case of a categorical (discrete) variable.

So, for instance, suppose a numerical predictor has the values `[1.0, 6.5, 3.5, NaN, 5.0]`. Then, you might replace the `NaN` with the mean of the known values, `[1.0, 6.5, 3.5, 5.0]`, which is 4.0.

**Exercise 1 (3 points)**: Create a function called **`imputevalue()`** that takes, as its inputs, a dataframe, the name of a column in that dataframe, and the replacement method. The replacement method will be a string, either `"mean"` or `"mode"`.

With these three inputs, your function should do the following:

1. Create a copy of the dataframe (i.e., the original should remain intact).
2. Compute the **mean** or **mode** of the column **without** the NaN values.
3. Replace the NaN's in that column with the computed mean/mode.
4. Return this new dataframe (i.e., not just the column containing the newly imputed values).

In [66]:
def imputevalue(df, col, func):
    assert func in ['mean', 'mode'], "You might have edited the assertion in this code cell, please reload this cell"
    df_imp = df.copy()
    if func == 'mean':
        col_mean = np.mean(df[col])
        df_imp[col] = df_imp[col].fillna(col_mean)
    else:
        col_mode = df_imp[col].mode()[0]
        df_imp[col] = df_imp[col].fillna(col_mode)
    return df_imp


,Predictor A,Predictor B,Predictor C,Predictor D,Predictor E,Predictor F,Predictor G,Response
0,30.83,G,1.250,1,t,202.0,Mrketing 1,1.0
1,58.67,J,3.040,6,t,43.0,Mkt6,1.0
2,24.50,B,1.500,0,f,280.0,M0,1.0
3,27.83,B,3.750,5,f,100.0,Marketing 5,1.0
4,20.17,B,1.710,0,t,120.0,Spend 0,1.0
5,32.08,M,2.500,0,f,360.0,Marketing 0,1.0
6,33.17,B,6.500,0,f,164.0,Marketing 0,1.0
7,22.92,A,0.040,0,f,80.0,Mrk 0,1.0
8,54.42,B,3.960,0,t,180.0,Marketing 0,1.0
9,42.50,A,3.165,0,f,52.0,Mrk 0,1.0


In [68]:
##Test cell: Exercise 1
pd.options.mode.chained_assignment = None

df2 = imputevalue(creditwithresponse, 'Predictor A', 'mean')
assert not(df2.equals(creditwithresponse)), 'You have not created a copy of the dataframe'
assert (round(np.mean(df2['Predictor A']), 2) >= 31.8) & (round(np.mean(df2['Predictor A']), 2)<=31.9), "The imputed value is incorrect. Please check your code"

df2 = imputevalue(creditwithresponse, 'Predictor B', 'mode')
assert df2.loc[:,'Predictor B'].mode()[0] == 'B', "The imputed value is incorrect. Please check your code"

credit_imputed_temp = imputevalue(creditwithresponse, 'Predictor A', 'mean')
credit_imputed = imputevalue(credit_imputed_temp, 'Predictor B', 'mode')

assert credit_imputed['Predictor A'].notnull().all()==True, 'There are still some missing values in Predictor A'
assert credit_imputed['Predictor B'].notnull().all()==True, 'There are still some missing values in Predictor B'

print("\n(Passed!)")


(Passed!)


Using the preceding techniques (removing missing rows or imputing values), we've covered all variables except `Predictor G`. Let's treat that one next. First, let's inspect it:

In [69]:
credit_imputed['Predictor G'].head()

0     Mrketing 1
1           Mkt6
2             M0
3    Marketing 5
4        Spend 0
Name: Predictor G, dtype: object

This column actually contains marketing expenditures in thousands of dollars. For example, `'Marketing 1'` means that a total of $1000 was spent on this marketing campaign.

As you can see, these data were not entered in a consistent way, except that a numerical value does appear. In this exercise you are required to extract the numbers from the column's values, e.g., extract **`1`** from `'Marketing 1'`.

Please note that the following facts about the values in the column 'Predictor G'.
1. Each value begins with a string of alphabetic characters. This string may vary from row to row.
2. A space may or may not follow that initial string of alphabetic characters.
3. The string ends with a sequence of digits.

Refer to the sample values from the call to `.head()` above.

**Exercise 2 (3 points)**: Create a function **`strip_text()`** that takes a **`(dataframe, column)`** as inputs and returns a **dataframe** according to the desciption below.

With these two inputs, your function should:

1. Create a copy of the dataframe, i.e., the original should remain intact.
2. For the given column, remove all the text in the column so that it contains only numbers (integers).
3. Return this new dataframe, i.e., not just the column containing the newly imputed values.

In [132]:
def strip_text(df, col):
    df_strip = df.copy()
    df_strip[col] = df_strip[col].str.extract('(\d+)').astype(int)
    return df_strip



instr = pd.DataFrame(['Rich0','Rachel    2', 'Sam123', 'Ben 012', 'Evan 999', 'Chinmay12', '   Raghav12'])
instr2 = instr.rename(columns={0:'col1'})
strip_text(instr2, 'col1')

,col1
0,0
1,2
2,123
3,12
4,999
5,12
6,12


In [133]:
##Test cell: Exercise 2

instr = pd.DataFrame(['Rich0','Rachel    2', 'Sam123', 'Ben 012', 'Evan 999', 'Chinmay12', '   Raghav12'])
instr2 = instr.rename(columns={0:'col1'})
assert strip_text(instr2,'col1').equals(pd.DataFrame([0,2,123,12,999,12,12]).rename(columns={0:'col1'})),"Please check your output by running your function on the 'instr' dataframe"

credit_cleaned = strip_text(credit_imputed,'Predictor G')
assert not(credit_cleaned.equals(credit_imputed)), 'You have not created a copy of the dataframe'
assert credit_cleaned['Predictor G'].dtype  == 'int64', "Output data type does not match"
assert len(credit_cleaned) == 612, "Your dataframe output is not of the appropriate length"
assert (round(np.mean(credit_cleaned['Predictor G']),2) >= 2.62) & (round(np.mean(credit_cleaned['Predictor G']),2)<=2.64), "The imputed data does not match. You could try replicating these tests on the 'instr' dataframe above."
assert (round(np.sum(credit_cleaned['Predictor G']),2) >= 1611.0) & (round(np.sum(credit_cleaned['Predictor G']),2) <= 1613.0) , "The imputed data does not match. You could try replicating these tests on the 'instr' dataframe above."

print("\n(Passed!)")


(Passed!)


Now that you have cleaned your dataset, let's do one final check to see if we still have any missing values.

In [134]:
credit_cleaned.isnull().any()

Predictor A    False
Predictor B    False
Predictor C    False
Predictor D    False
Predictor E    False
Predictor F    False
Predictor G    False
Response       False
dtype: bool

You should see all `False` values, meaning there is no missing data in any of the columns. If so, great!

**Creating Interaction Terms in the Data**

Sometimes, for analysis purposes, it is better to create _interaction predictors_, which are new predictors that modify or combine existing predictors. For example, in a marketing scenario, spending on TV marketing might have a quadratic relationship with the sales of the product. We would hence want to include ** $(\mathrm{TV\ marketing})^2$ **  as a predictor to better capture the relationship.

In this final exercise we will create a new predictor that is a combination of the predictors in the dataset **`credit_cleaned`**.

**Exercise 3 (3 points):** Create a function **`familiarity()`** that takes as its inputs a dataframe (`df`), the names of three input columns (`column1`, `column2`, `column3`), and the name of a new output column (`columnnew`). It should compute for the values of this new column what appears in the formula below.

**$$\mathtt{columnnew} = \frac{\mathtt{column1}}{e^{\mathtt{column2}}} - \sqrt{\mathtt{column3}},$$**

where **$$\sqrt{\mathtt{column3}} = (\mathtt{column3})^{0.5}.$$**

The return value for the function will be a dataframe with the new column, **`columnnew`**, **in addition** to all the original columns in the dataframe.

> **Note.** If a value in column 3 is negative, so that the square-root is undefined, set the corresponding value in `columnnew` to zero (0).

In [157]:
import math
def familiarity(df, column1, column2, column3, columnnew):
    df_fam = df.copy()
    df_fam[columnnew] = ((df_fam[column1]) / np.exp(df_fam[column2])) - df_fam[column3]**.5
    df_fam[columnnew] = df_fam[columnnew].fillna(0)
    return df_fam


In [158]:
##Test cell: Exercise 3

d={'col1':[1,2,3,4,5], 'col2':[2,3,4,0,4], 'col3':[-9,2,-8,0,0]}
df = pd.DataFrame(d)
dffamiliarity = familiarity(df, 'col1', 'col2', 'col3', 'colnew')
assert dffamiliarity.loc[dffamiliarity['col3']<0,'colnew'].all()==0, "The non negative case for col3 is failing. Please check your code"


credit_final = familiarity(credit_cleaned, 'Predictor A', 'Predictor C', 'Predictor G', 'Predictor H')
assert 'Predictor H' in credit_final, "Column 'Predictor H' does not exist"
assert len(credit_final) == 612, "The length of the dataframe does not match the required length"
assert (round(np.sum(credit_final['Predictor H']),2) >= 7262.4) & (round(np.sum(credit_final['Predictor H']),2) <= 7262.5), "The sum of values in Predictor H do not match the required vlue"

print("\n(Passed!)")


(Passed!)


At this point, you have completed all the exercises and can go ahead and submit the notebook!

However, we have however added a small piece of code below to give you an idea of how simple it is to create a predicitive model in Python. It is **not graded** and hence you can submit this notebook, complete other notebooks and come back and have a look at it!

In [160]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#Split the dataset into predictors and response
datax = credit_final[['Predictor A', 'Predictor C', 'Predictor D', 'Predictor F', 'Predictor G', 'Predictor H']]
datay = credit_final[['Response']]

#Traintest split - Test sets are created to test the accuracy of your model on a piece of data that is not used to train the model
X_train, X_test, y_train, y_test = train_test_split(datax, np.ravel(datay), test_size=0.20, random_state=42)

forest = RandomForestClassifier(n_estimators=500) #Number of decision trees in the forest = 500
forest.fit(X_train,y_train) #Train the classifier using the train data
forest_pred = forest.predict(X_test) #Predict the classes for the test data
print("The testing accuracy of the random forest classifier is: ",accuracy_score(y_test, forest_pred)) #Print the accuracy of the model

The testing accuracy of the random forest classifier is:  0.7398373983739838


The accuracy of the model above is about 70% or so. Due to the nature of the dataset being artificial, we don't expect a higher accuracy. Instead, our purpose here is to give you an idea as to how easy it is to do predictive modelling (machine learning) in Python.


**Fin!** That's the end of this problem. Don't forget to restart and run this notebook from the beginning to verify that it works top-to-bottom before submitting. You can move on to the next problem